In [1]:
import numpy
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import random
import networkx as nx
import plotly.graph_objects as go
import plotly.offline as pyo
import pickle
import os

In [2]:
data = pd.read_csv("../Data/candidates_ge15.csv")

In [3]:
Malaysian_politicians = []

for index, row in data.iterrows():
    Malaysian_politicians.append((row["name"], row["party"]))

In [4]:
G = nx.Graph()
parties = set()  # Keep track of all unique parties

for name, party in Malaysian_politicians:
    G.add_node(name, type="politician", party=party)
    G.add_node(party, type="party")
    G.add_edge(name, party)
    parties.add(party)

In [5]:
file_exists = os.path.isfile("pos.pkl")
if file_exists:
    with open("pos.pkl", "rb") as f:
        pos = pickle.load(f)
else:
    # Extract positions for nodes in the graph
    pos = nx.spring_layout(G, iterations=200)
    
    with open("pos.pkl", "wb") as f:
        pickle.dump(pos, f)

In [6]:
node_x = []
node_y = []
text = []
node_sizes = []
node_colors = []

# Define colors or other visual cues for parties and politicians
party_color = "red"  # Example color for parties
politician_color = "blue"  # Example color for politicians

for node in G.nodes():
    x, y = pos[node]
    node_x.append(x)
    node_y.append(y)
    text.append(node)

    if G.nodes[node]["type"] == "party":
        node_sizes.append(
            max(len(data[data["party"] == node])*200 / len(data),10)
        )  # Larger size for party nodes
        node_colors.append(party_color)
    else:
        node_sizes.append(0.5)  # Smaller size for politician nodes
        node_colors.append(politician_color)

node_trace = go.Scatter(
    x=node_x,
    y=node_y,
    mode="markers+text",
    hoverinfo="text",
    text=text,
    marker=dict(size=node_sizes, color=node_colors, line_width=1),
)

edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(
    x=edge_x,
    y=edge_y,
    line=dict(width=0.5, color="#888"),
    hoverinfo="none",
    mode="lines",
)

# Create the figure
fig = go.Figure(
    data=[edge_trace, node_trace],
    layout=go.Layout(
        title="Network graph of Malaysian Politicians and Parties",
        titlefont_size=16,
        showlegend=False,
        hovermode="closest",
        margin=dict(b=20, l=5, r=5, t=40),
        xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
        yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
    ),
)

# Show the figure
pyo.plot(fig)

'temp-plot.html'